# Company Name matching
### Problème
Lorsqu'un prospect rempli un formulaire, il saisit le nom de son entreprise. Afin de récolter d'avantage d'informations sur le prospect, il est nécessaire d'identifier l'entreprise, c'est à dire de connaitre son numéro SIREN. Cependant, un matching naif entre la saisie utilisateur et une base de référence Nom-Siren n'est pas satisfaisant.
En effet la saisie utilisateur est imparfaite (ne correspond à aucun nom de la base de référence):
* Erreurs de saisie
* Noms non légaux :
  * Abbreviations
  * Département, région
  * Surnom
  * Inversion

Quelques exemples :  

In [47]:
#! TODO : Ecrire exemple

### Solution 
L'idée est donc de construire un **moteur de recherche**, qui à partir d'une requête (ici saisie du champ companyName) de l'utilisateur retrouve le document (ici le nom *standard* dans la base de référence), ce qui nous permettra de faire le lien entre la saisie du nom de l'entreprise et son numéro de SIREN.

Pour ce faire, plusieurs pistes ont été explorées:
* standardisation du nom puis matching naif
* utilisation d'un moteur de recherche déjà entrainé
Effectuer une requete sur google (ou autre) de la forme `companyName site:"societe.com"` et récupérer le premier résultat renvoyé. Cependant, le nombre de requêtes google est limitée (10-20/heure sans astuce, 200/heure avec), ce qui ne permet pas de tester efficacement sur la base de données historique. Egalement, societe.com n'autorise pas l'utilisation de scrapping à des fins commerciales. Pour ce qui est de la première limitation, il pourrait être envisageable d'utiliser cette méthode en production, car le flux de requêtes à effectuer est relativement faible. Pour la deuxième, utiliser l'addresse url du site suffit car elle est sous la forme `https://www.societe.com/societe/companyName-SIREN.html`. 
* Deux étapes :
  1. Tokeization et identification des stop words correspondants à des mots communs qui n'ajoutent pas d'information sur les entreprises
  2. Identification du nom standard (dans la base de référence) 

C'est cette deuxième solution que nous avons trouvé la plus pertinente et la plus précise.

In [1]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
from collections import Counter

## Chargement des bases de données
Deux bases de données différentes sont chargées : une base test, qui simule l'entrée utilisateur, et la base de référence, qui permet de faire le lien entre un nom standard de référence pour l'entreprise et son numéro SIREN. Ces données sous fichiers textes sont chargées dans des dataframes pandas.   
* **test_inputs** : On récupère les 500 plus grandes entreprises de France, collectée par un copier coller dans un fichier texte  
* **siren_df** : données SIREN-noms issues de l'api data gouv

In [2]:
df = pd.read_csv("../data/top500verifCom.tsv",
                     names=["companyName", "postCode", "city", "CA"], sep="\t")
test_inputs = df[["companyName"]]
print(f"Taille du test set : {len(test_inputs)}")
test_inputs.head()

Taille du test set : 500


,companyName
0,PSA AUTOMOBILES SA
1,ELECTRICITE DE FRANCE
2,AIRBUS
3,RENAULT SAS
4,ORANGE


In [3]:
siren_table_path = "../data/siren_table.csv"
siren_df = pd.read_csv(siren_table_path, dtype={"Siren": "object"})

print(f"Taille de la base de données de référence : {len(siren_df)}")
siren_df.head()

Taille de la base de données de référence : 1213321


,companyName,Siren
0,TA.MI.RO.SA. HOLDING,802504068
1,M.T.M.,945451169
2,SOCIETE CAPBRETONNAISE DE DISTRIBUTION SOCADI,987020203
3,SOCIETE INTERGRAS,997868807
4,NANTET MENUISERIE,76520279


## Cleaning
En examinant la base de données, les charactères et les motifs *polluants* ont été supprimé.

In [4]:
def clean_spaces(text):
    text=text.replace('  ', ' ')
    text=text.strip()
    if len(text) < 1:
        return "unmeaningfulofatext"
    return text

def prep(company):
    """
    Clean un nom d'entreprise saisie par un utilisateur

    Arguments:
        company {string} -- nom de l'entreprise, correspond au champ companyName de la table Lead

    Returns:
        string -- le nom de l'entreprise, avec des caractères polluants en moins.
    """
    company = company.encode("ascii", errors="ignore").decode()
    company = clean_spaces(company)
    company = company.lower()
    chars_to_remove = [")", "(", ".", "|", "[", "]", "{", "}", "'", ",", ";"]
    rx = '[' + re.escape(''.join(chars_to_remove)) + ']'
    company = re.sub(rx, '', company)
    company = re.sub(r'[0-9]+', '', company)
    return company
prep("BNP  PPAri,bas")


'bnp pparibas'

Application du preprocessing aux colonnes des deux tables et matching "un à un", parfait

In [5]:
siren_df["companyNameClean"] = siren_df["companyName"].map(prep)
test_inputs["companyNameClean"] = test_inputs["companyName"].map(prep)

naif_result = test_inputs.merge(siren_df, on="companyNameClean", how="left")
missing_joins = naif_result["Siren"].isnull().sum()/len(test_inputs)*100 
#un nom d'entreprise n'ayant pas de numéro siren dans la table naif_result n'apparait qu'une fois

print(f'{missing_joins:.2f} % de joins manquants sur les {len(test_inputs)} champs du test set. La jointure donne {len(naif_result)} champs')

C:\Users\benja\AppData\Local\Temp\ipykernel_25232\804919759.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_inputs["companyNameClean"] = test_inputs["companyName"].map(prep)


23.60 % de joins manquants sur les 500 champs du test set. La jointure donne 575 champs


In [6]:
naif_result[naif_result["Siren"].isnull()]

,companyName_x,companyNameClean,companyName_y,Siren
1,ELECTRICITE DE FRANCE,electricite de france,NaN,NaN
14,TOTALENERGIES MARKETING FRANCE,totalenergies marketing france,NaN,NaN
16,LA FRANCAISE DES JEUX,la francaise des jeux,NaN,NaN
21,SYSTEME U CENTRALE NATIONALE,systeme u centrale nationale,NaN,NaN
24,LA POSTE 75015,la poste,NaN,NaN
...,...,...,...,...
544,TOLL COLLECT GMBH,toll collect gmbh,NaN,NaN
547,DAHER AEROSPACE,daher aerospace,NaN,NaN
559,SOCIETE VITREENNE D'ABATTAGE,societe vitreenne dabattage,NaN,NaN
561,AB INBEV FRANCE,ab inbev france,NaN,NaN


## A étudier plus tard : noms non unique dans chiffre-cle-2020
--> faire à la main pour les 500 plus grosses entreprises
--> attention au merge

Lors de la jointure, des noms d'entreprises correspondent à différents SIREN à droite. En effet, après preprocessing, les noms d'entreprise perdent leur unicité dans siren_table

In [7]:
g = naif_result.groupby("companyNameClean").size()
g.where(g>1).dropna()

companyNameClean
amadeus                                              4.0
as                                                  13.0
autodistribution                                     2.0
axione                                               2.0
bel                                                 13.0
boulanger                                            4.0
chanel                                               3.0
clinea                                               2.0
cora                                                 5.0
csf                                                  4.0
elivia                                               2.0
la halle                                             3.0
orange                                               3.0
orpea                                                2.0
pomona                                               2.0
prosol                                               4.0
roche                                               10.0
sca fruits leg

Des entreprises apparaissent de nombreuses fois : après cleaning, leur nom n'est plus unique.

In [8]:
naif_result[naif_result.duplicated(subset=["companyNameClean"], keep=False)].sample(10)

,companyName_x,companyNameClean,companyName_y,Siren
396,ORPEA,orpea,ORPEA,388609117
103,AS 24,as,AS 2,813620184
6,ORANGE,orange,ORANGE,485070205
283,BEL,bel,BEL,849621776
64,CORA,cora,CORA,519084271
523,CLINEA,clinea,CLINEA,301160750
210,SOCIETE COOPERATIVE D'APPROVISIONNEMENT PARIS-EST,societe cooperative dapprovisionnement paris-est,SOCIETE COOPERATIVE D'APPROVISIONNEMENT PARIS-EST,301986154
412,SYNERGIE,synergie,SYNERGIE,750745531
323,SCA FRUITS LEGUMES FLEURS,sca fruits legumes fleurs,SCA FRUITS LEGUMES FLEURS,353402779
145,CHANEL,chanel,CHANEL,833854235


In [9]:
result_df_2 = naif_result.merge(siren_df, left_on="companyName_x", right_on="companyName", suffixes=("_a", "_b"))

In [10]:
result_df_2[result_df_2["Siren_a"]==result_df_2["Siren_b"]]

,companyName_x,companyNameClean_a,companyName_y,Siren_a,companyName,Siren_b,companyNameClean_b
0,AIRBUS,airbus,AIRBUS,383474814,AIRBUS,383474814,airbus
1,ORANGE,orange,ORANGE,879144723,ORANGE,879144723,orange
5,ORANGE,orange,ORANGE,791052525,ORANGE,791052525,orange
9,ORANGE,orange,ORANGE,485070205,ORANGE,485070205,orange
12,CSF,csf,CSF,479228306,CSF,479228306,csf
...,...,...,...,...,...,...,...
766,CASTEL FRERES,castel freres,CASTEL FRERES,482283694,CASTEL FRERES,482283694,castel freres
767,INTERFORUM,interforum,INTERFORUM,612039073,INTERFORUM,612039073,interforum
768,AXIONE,axione,AXIONE,449586544,AXIONE,449586544,axione
771,AXIONE,axione,AXIONE,479787400,AXIONE,479787400,axione


## NLP
On utilise la méthode de fuzzy string matching [[Ukkonnen](https://www.sciencedirect.com/science/article/pii/S0019995885800462)], que l'on applique avec les stop words judicieusement définis pour les noms d'entreprise.

### Calcul du score de similarités
Dans un premier temps, il s'agit de vectoriser chaque nom d'entreprise, à partir des tokens définis par la méthode ngram

In [11]:
def ngrams(string, n=3):
    ngrams = zip(*[string[i:] for i in range(n)])
    return [''.join(ngram) for ngram in ngrams]
ngrams("benjamin")

['ben', 'enj', 'nja', 'jam', 'ami', 'min']

In [13]:
def knn_reference(standard_names=[], k_matches=5, ngram_length=3):
    #Tf Idf matrice à partir des données référece
    vectorizer = TfidfVectorizer(min_df=1, analyzer=lambda word : ngrams(word, ngram_length))
    tf_idf_ref = vectorizer.fit_transform(standard_names)

    # Fit le k-NN sur cette matrice
    neighbors = NearestNeighbors(n_neighbors=k_matches, n_jobs=-1, metric="cosine").fit(
        tf_idf_ref
    )
    return neighbors, vectorizer

La liste standard contient les noms standards de référence. 
Chaque nom est découpé en ngrams.
On transforme la liste de ngram dans une sparce matrix [tf-idf](https://medium.com/@cmukesh8688/tf-idf-vectorizer-scikit-learn-dbc0244a911a)
A partir de cette matrice, on entraine un k-nn
On transforme la liste test dans une matrice.
Calcul les distance de tous les n-grams matchs les plus proches
Calcul un match score

In [14]:



def matcher(test_names=[], standard_names=[], k_matches=5, ngram_length=3):
    """Pour chaque entrée dans la liste test_names, renvoie
    les k premiers matches de la liste standard.
    
    Arguments:
        test_names {string list} -- noms de l'entreprise test, saisies par l'utilisateur,
                                que l'on veut faire match avec un nom standard, déjà clean
        standard_names {string list} -- noms standards des entreprises, déjà clean
        k-matches {int} -- nombre de matchs à renvoyer
        ngram_length -- longueur des ngrams

    Returns:
        DataFrame -- avec la liste originale, et `k_matches`c colonnes qui contiennet
                    les matchs les plus proche dans `standard` et leurs scores de similarité
    """

    neighbors = knn_reference(standard_names, k_matches, ngram_length)
    
    # Prediction du test set avec ce modèle
    tf_idf_test_names = vectorizer.transform(test_names)
    distances, standard_indices = neighbors.kneighbors(tf_idf_test_names)
    
    # Extract top Match Score (which is just the distance to the nearest neighbour),
    # Original match item, and Lookup matches.
    test_names_result = []
    confidences = []
    indexes = []
    standard_names_result = []
    
    for i, lookup_index in enumerate(standard_indices):
        test_name = test_names[i]
        # lookup names in lookup list
        refs= [standard_names[index] for index in lookup_index]
        # transform distances to confidences and store
        confidence = [1 - round(dist, 2) for dist in distances[i]]
        test_names_result.append(test_name)
        # store index
        indexes.append(lookup_index)
        confidences.append(confidence)
        standard_names_result.append(refs)

    # Convert to df
    df_orig_name = pd.DataFrame(test_names_result, columns=["testCompanyName"])

    df_lookups = pd.DataFrame(
        standard_names_result, columns=["Res " + str(x + 1) for x in range(0, k_matches)]
    )
    df_confidence = pd.DataFrame(
        confidences,
        columns=["Res " + str(x + 1) + " Confidence" for x in range(0, k_matches)],
    )
    df_index = pd.DataFrame(
        indexes,
        columns=["Res " + str(x + 1) + " Index" for x in range(0, k_matches)],
    )

    matches = pd.concat([df_orig_name, df_lookups, df_confidence, df_index], axis=1)

    # Reordonnecement des colonnes
    lookup_cols = list(matches.columns.values)
    lookup_cols_reordered = [lookup_cols[0]]
    for i in range(1, k_matches + 1):
        lookup_cols_reordered.append(lookup_cols[i])
        lookup_cols_reordered.append(lookup_cols[i + k_matches])
        lookup_cols_reordered.append(lookup_cols[i + 2 * k_matches])
    matches = matches[lookup_cols_reordered]
    return matches

In [15]:
matcher(test_names=test_inputs["companyNameClean"].values, standard_names=siren_df['companyNameClean'].values)

NameError: name 'vectorizer' is not defined